In [1]:
import numpy as np
indices = np.random.choice(10, 5, replace=True)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(arr[indices])
print(arr[-11:])
reward_buffer = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1]
size = 7
game_start_index = 0
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
game_start_index = 7
size = 15
print(len(reward_buffer))
reward = reward_buffer[size - 1]
updated_rewards = np.empty((size - game_start_index,), int)
updated_rewards[::2] = 1 * reward
updated_rewards[1::2] = -1 * reward
updated_rewards = np.flip(updated_rewards)
print(updated_rewards)
reward_buffer[game_start_index:size] = updated_rewards
print(reward_buffer)
print(len(reward_buffer))
overflow_reward = 1
overflow_count = 1

[ 2 10  1 10  4]
[ 1  2  3  4  5  6  7  8  9 10]
[ 1 -1  1 -1  1 -1  1]
[1, -1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, -1]
15
[ 1 -1  1 -1  1 -1  1 -1]
[1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1]
15


In [2]:
# reward_buffer = np.array([1, -1, 1, -1, 1, -1, 0])
# reward_buffer[6] = [1, 1, 1, 1]
# print(reward_buffer)

In [12]:
from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf
import environments

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# as recommended by the original paper, should already include max pooling
env = gym.make("environments_/TicTacToe-v0")


config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_hidden_filters': 256,
        'critic_dense_size': 256,
        'actor_hidden_filters': 256,
        'actor_dense_size': 256,
        'memory_size': 10000,
        'max_game_length': 1000,
        'batch_size': 10,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 1,
        'monte_carlo_simulations': 10,
        'two_player': False,
    }    

agent = AlphaZeroAgent(env, config=config)

NamespaceNotFound: Namespace environments_ not found. Have you installed the proper package for environments_?

In [ ]:
agent.train()

In [ ]:
array = np.zeros((3, 9))
array[0] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(array)